In [ ]:

from pyspark.sql import SparkSession
from silver_properties import SilverProcessor
spark = SparkSession.builder.appName("SilverLayer").getOrCreate()

bronze_table = "br_properties"


silver_processor = SilverProcessor(spark, bronze_table)

try:
    import dlt
except ImportError:
    class dlt:
        def table(comment, **options):
            def _(f): return f
            return _
        def read_stream(table_name):
            return spark.readStream.table(table_name)


@dlt.table(
    comment="Quarantine records for silver layer",
    table_properties={'quality': 'silver'},
    partition_cols=['is_quarantined']
)
def si_quarantine_records():
    bronze_df = dlt.read_stream(bronze_table)
    valid_df, invalid_df = silver_processor.process_data(bronze_df)
    return invalid_df.dropDuplicates()

@dlt.table(
    comment="Transaction fact table",
    table_properties={'quality': 'silver'}
)
def si_fact_transaction():
    bronze_df = dlt.read_stream(bronze_table)
    valid_df, _ = silver_processor.process_data(bronze_df)
    return silver_processor.transform_transaction_facts(valid_df).dropDuplicates()

@dlt.table(
    comment="Property dimension table",
    table_properties={'quality': 'silver'}
)
def si_dim_properties():
    bronze_df = dlt.read_stream(bronze_table)
    valid_df, _ = silver_processor.process_data(bronze_df)
    return silver_processor.transform_property_dimensions(valid_df).dropDuplicates()